- [ ] seperate intl and domestic departure count 
- [ ] seperate A320 and Atr count

In [2]:
%%time
import datetime
import pickle
import re

import gspread
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from altair.vegalite.v4.schema.channels import Tooltip
import altair as alt
import math


gc = gspread.service_account(filename='./crudg.json')
sheet = gc.open("flightsched").get_worksheet(3)
dataframe = pd.DataFrame(sheet.get_all_records())



Wall time: 5.23 s


In [4]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28956 entries, 0 to 28955
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   A/C No      28956 non-null  int64 
 1   Flt Desg    28956 non-null  int64 
 2   Freq        28956 non-null  int64 
 3   Dept Arp    28956 non-null  object
 4   Dept Time   28956 non-null  object
 5   Arvl Arp    28956 non-null  object
 6   Arrv Time   28956 non-null  object
 7   Equip       28956 non-null  object
 8   Block Time  28956 non-null  object
 9   Eff Date    28956 non-null  object
 10  Dis Date    28956 non-null  object
dtypes: int64(3), object(8)
memory usage: 2.4+ MB


In [ ]:
import IPython
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
dataframe["engg_dep"]  = pd.to_datetime((dataframe["Eff Date"].astype(str) + "-" + dataframe["Dept Time"].astype(str)) , format='%d-%m-%y-%h:%M',
                                   infer_datetime_format=True,utc=False )

dataframe["aocs_arr"] = pd.to_datetime((dataframe["Eff Date"].astype(str) + "-" + dataframe["Arrv Time"].astype(str)) , format='%d-%m-%y-%h:%M',
                                   infer_datetime_format=True,utc=False )


dataframe = dataframe.drop(columns  = ['Dept Time', 'Arrv Time',  'Eff Date',  'Dis Date',"Freq"])
                                        
dataframe.head()

In [ ]:
testpd = dataframe.copy()
testpd = testpd.loc[testpd["engg_dep"].dt.month == 5]


In [ ]:
testpd.head(30)

In [ ]:
# adding column "engg_arr" and "deptype"
testpd["engg_arr"] = np.nan
testpd["deptype"] = np.nan
testpd["cal_date"] = testpd["engg_dep"].dt.day

# making block time as time delta in python
testpd["Block Time"] = testpd["Block Time"].astype("str")
splitString = testpd["Block Time"].str.split(':', expand=True).astype(int)
testpd["Block Time"] = pd.to_timedelta(
    splitString[0], unit='h') + pd.to_timedelta(splitString[1], unit='m')

testpd = testpd[['Dept Arp', 'engg_arr', 'Flt Desg', 'A/C No', 'Equip', 'Arvl Arp','engg_dep', 'aocs_arr', 'deptype', 'Block Time', 'cal_date']]


In [ ]:
x_df = testpd.loc[(testpd["A/C No"] == 1) &
                               (testpd["cal_date"] == 1)]
x_df = x_df.head(1)
x_df

In [ ]:
def addlastRowArrDep(paramdf):
    converted = paramdf.to_numpy()
    emptyarray = np.full((1, 11), np.nan)
    if (converted.shape[0] == 1):
        temparray = converted
        return temparray
    else:
        temparray = np.append(converted, emptyarray, axis=0)
        lentemparray = temparray.shape[0]
        temparray[lentemparray - 1][0:2] = temparray[lentemparray - 2][5:7]
        temparray[lentemparray - 1][2:5] = temparray[lentemparray - 2][2:5]
        for xray in range(0, lentemparray):
            if xray == 0:
                pass
               
            elif xray == lentemparray - 1:
                pass
               
            else:
                temparray[xray][1] = temparray[xray - 1][6]
                
        return temparray
    

In [ ]:
masterList = []
ninedf = addlastRowArrDep(x_df)
[masterList.append(zulu) for zulu in ninedf]
cleanDf = pd.DataFrame(masterList,
                       columns = ['Dept Arp', 'engg_arr', 'Flt Desg', 'A/C No', 'Equip', 'Arvl Arp','engg_dep', 'aocs_arr', 'deptype', 'Block Time', 'cal_date'])
cleanDf.head(30)

In [ ]:
def masterDfMaker(paramdf):

    # list of Comment List which are unique
    textCommentList = paramdf["A/C No"].unique()
    # list(array of all days in our data )
    uniqueDates = np.sort(testpd["engg_dep"].dt.day.unique())

    masterList = []
    for xray in textCommentList:
        for alpha in uniqueDates:
            # filtering dataframe on CommentValue and uniqueDateValue
            x_df = paramdf.loc[(paramdf["A/C No"] == xray) &
                               (paramdf["cal_date"] == alpha)]
            # check if dataframe is empty
            if len(x_df) == 0:
                continue
            # sorting by engg_dep for safety
            x_df = x_df.sort_values(by=["engg_dep"], ascending=True)
            x_df = addlastRowArrDep(x_df)
            [masterList.append(zulu) for zulu in x_df]
    return masterList

In [ ]:
masterList = masterDfMaker(testpd)

In [ ]:
cleanDf = pd.DataFrame(masterList,
                       columns = ['Dept Arp', 'engg_arr', 'Flt Desg', 'A/C No', 'Equip', 'Arvl Arp','engg_dep', 'aocs_arr', 'deptype', 'Block Time', 'cal_date'])
cleanDf.head(1)

In [ ]:
cleanDf = cleanDf.copy()


# adding columns cal_date and transit_time
# already added before during refactor
# cleanDf["cal_date"]= np.nan
cleanDf["transit_time"] = np.nan
# avoidy loc error hency copy
cleanDf = cleanDf.copy()
# cal_date calue for base dep/ pd isnull for time is something new
cleanDf.loc[pd.isnull(cleanDf["engg_arr"]),
            "cal_date"] = cleanDf["engg_dep"].dt.date
# cal_date calue for base arr/
cleanDf.loc[pd.isnull(cleanDf["engg_dep"]),
            "cal_date"] = cleanDf["engg_arr"].dt.date
# cal_date calue for transit using the notnull operator
cleanDf.loc[(pd.notnull(cleanDf["engg_arr"]) & pd.notnull(cleanDf["engg_dep"])),
            "cal_date"] = cleanDf["engg_dep"].dt.date
# departuretype as basedep
cleanDf.loc[pd.isnull(cleanDf["engg_arr"]), "deptype"] = "baseDep"
#departure type as baseArr
cleanDf.loc[pd.isnull(cleanDf["engg_dep"]), "deptype"] = "baseArr"
#deparure type as transit
cleanDf.loc[(pd.notnull(cleanDf["engg_arr"]) & pd.notnull(cleanDf["engg_dep"])),
            "deptype"] = "Transit"
# setting type in a column cal_date
cleanDf["cal_date"] = cleanDf["cal_date"].astype("datetime64[ns]")
# calculating transit time
cleanDf.loc[cleanDf['deptype'] == "Transit",
            "transit_time"] = cleanDf["engg_dep"] - cleanDf["engg_arr"]
# setting transitime column type
cleanDf["transit_time"] = cleanDf["transit_time"].astype("timedelta64[ns]")
cleanDf= cleanDf.astype({"deptype":"category","Dept Arp":"category","Arvl Arp":"category","A/C No":"category","Equip":"category","engg_arr":"datetime64[ns]","engg_dep":"datetime64[ns]","Flt Desg":"category","Block Time":"timedelta64[ns]"})

# observed improvement of from 4.3 mb to 2.3 mb after setting of category 

In [ ]:
cleanDf.info()

In [ ]:
def getMonthChart(dataframe: pd.DataFrame,stationList: list = None) -> alt.vegalite.v4.api.VConcatChart:
    """[summary]

    Args:
        month ([type]): [description]
        station ([type]): [description]

    Returns:
        [type]: [description]
    TODO : chart height claculation based on 
    """
    # checking Area
    # test-One ensuring the type of keyword parameter and throwing error
    if stationList != None:
        assert isinstance(stationList,list) ,'stationList should be a (pythonType(List))'
    # ensuring argument is of type pandas datafrme  
    assert isinstance(dataframe,pd.DataFrame),"dataframe should be of type #pandas Dataframe #"
        
        

    groupedDf = dataframe.copy()
    groupedDf = groupedDf.groupby(["cal_date","deptype","Dept Arp"]).size().reset_index(name='count')
    
    if stationList != None:
        # test three if stationList is subset of avilable stations
        assert set(stationList).issubset(set(dataframe["Dept Arp"].unique())),"statonlist station not in dataframe['Dept Arp']"
        
        groupedDf = groupedDf.loc[groupedDf["Dept Arp"].isin(stationList)]
      
    listOperationsType = groupedDf["deptype"].unique().tolist()
    # claulating chart size for chart Height 
    
    chartHeight = math.ceil(len(groupedDf["Dept Arp"].unique())/5) *100
    
    # base Chart 
    base = alt.Chart().encode(alt.X('monthdate(cal_date):O', title='Days Of The Month',scale=alt.Scale(paddingInner=0)
                ),
        alt.Y('Dept Arp:N', title='Airport',scale=alt.Scale(paddingInner=0)))


    # rect chart 
    alpha = base.mark_rect().encode(
        color = alt.Color("count:Q",
        legend=alt.Legend(direction='horizontal')),
        
        tooltip=[ alt.Tooltip("cal_date",title='Date'),alt.Tooltip('count',title='No-of-Ops'),
                alt.Tooltip("deptype",title='Type-Ops'), alt.Tooltip("Dept Arp",title='Aiport')])
    
    # text chart 
    beta = base.mark_text( align='center', baseline='middle').encode(text='count:Q')

    # chart with data as one of Transit . base Dep , base base Arr
    chartOne = alt.layer(alpha, beta,data=groupedDf.loc[groupedDf["deptype"] == listOperationsType[0]].copy()).properties(
        title=str(listOperationsType[0])+' Ops in Station',
        width=1000,
    height=chartHeight).interactive()
    # chart with data as one of Transit . base Dep , base base Arr
    chartTwo = alt.layer(alpha, beta,data=groupedDf.loc[groupedDf["deptype"] == listOperationsType[1]].copy()).properties(
        title=str(listOperationsType[1])+' Ops in Station',
        width=1000,
    height=chartHeight).interactive()
    # chart with data as one of Transit . base Dep , base base Arr
    chartThree = alt.layer(alpha, beta,data=groupedDf.loc[groupedDf["deptype"] == listOperationsType[2]].copy()).properties(
        title= str(listOperationsType[2])+' Ops in Station',
        width=1000,
    height=chartHeight).interactive()
    return alt.vconcat(chartOne,chartTwo,chartThree)
    
    




In [ ]:
finalChart = getMonthChart(cleanDf)

In [ ]:
finalChart